# EuroPython 2018, Edinburgh – Stefan Behnel

See http://consulting.behnel.de/

In [2]:
%load_ext cython

In [5]:
import sys
import Cython
import numpy as np
import subprocess, os
print("Python %d.%d.%d %s %s" % sys.version_info)
print("Cython %s" % Cython.__version__)
print("NumPy  %s" % np.__version__)
print(subprocess.check_output([os.environ.get('CC', 'cc'), "--version"]).decode().splitlines()[0])
print([line for line in subprocess.check_output([os.environ.get('CC', 'cc'), "--version", "-v"],
                                                stderr=subprocess.STDOUT).decode().splitlines()
       if ' version ' in line][0])

Python 3.7.0 final 0
Cython 0.28.4
NumPy  1.15.0
cc (Ubuntu 5.4.0-6ubuntu1~16.04.10) 5.4.0 20160609
gcc version 5.4.0 20160609 (Ubuntu 5.4.0-6ubuntu1~16.04.10) 


## Cython Intro

In [6]:
from math import sin
sin(5)

-0.9589242746631385

In [3]:
%%cython
from math import sin
print(sin(5))

-0.9589242746631385


In [4]:
%%cython
cimport libc.math
sin_func = libc.math.sin

In [5]:
sin_func(5)

-0.9589242746631385

In [6]:
%%cython
cimport libc.math

def csin(double x):
    return libc.math.sin(x)

In [7]:
csin(5)

-0.9589242746631385

In [8]:
%%cython -a
cimport libc.math

def csin(double x):
    return libc.math.sin(x)

In [9]:
%%cython
cimport libc.math

def square_sin(double x):
    cdef double x_square = x * x
    return libc.math.sin(x_square)

In [10]:
square_sin(5)

-0.13235175009777303

## Exercise: Compile Python functions

Add more calculations to the Python function, but make sure they compile to plain C.

In [11]:
%%cython -a
# copy and extend ...

## Everyone likes taxes !

Idea borrowed from Caleb Hattingh, PyCon-AU 2015,
http://pyvideo.org/pycon-au-2015/easy-wins-with-cython-fast-and-multi-core.html

In [12]:
PEOPLE = 44_000_000
AVERAGE = 3703*12
print("Average income of {:,d} earners, Deutschland 2016: {:,d} €".format(PEOPLE, AVERAGE))

Average income of 44,000,000 earners, Deutschland 2016: 44,436 €


In [13]:
# lacking offical data, let's create some alternative facts
import numpy as np
mu, sigma = 10.64, .35
s = np.random.lognormal(mu, sigma, PEOPLE // 20)
['{:,.2f} €'.format(x) for x in (np.min(s), np.mean(s), np.max(s))]

['7,516.98 €', '44,418.12 €', '260,230.03 €']

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
count, bins, ignored = plt.hist(s[s < 110000], 100, normed=True, align='mid')
x = np.linspace(min(bins), max(bins), 101)
pdf = (np.exp(-(np.log(x) - mu)**2 / (2 * sigma**2))
       / (x * sigma * np.sqrt(2 * np.pi)))
plt.plot(bins, pdf, linewidth=2, color='r')
plt.axis('tight')
plt.show()

## Let's calculate everyone's taxes

https://de.wikipedia.org/wiki/Einkommensteuer_%28Deutschland%29#Tarif_2017

In [15]:
# from Wikipedia:
# =WENN(A1>256303; A1*0,45-16164,53;
#  WENN(A1>54057; A1*0,42-8475,44;
#  WENN(A1>13769; (A1-13769)*((A1-13769)*0,0000022376+0,2397)+939,57;
#  WENN(A1>8820; (A1-8820)*((A1-8820)*0,0000100727+0,14); 0))))

def calculate_tax(income):
    if income > 256303:
        return income * 0.45 - 16164.53
    elif income > 54057:
        return income * 0.42 - 8475.44
    elif income > 13769:
        return (income - 13769) * ((income - 13769) * 0.0000022376 + 0.2397) + 939.57
    elif income > 8820:
        return (income - 8820) * ((income - 8820) * 0.0000100727 + 0.14)
    else:
        return 0

def average_income(incomes):
    return sum(incomes) / len(incomes)

def average_tax_rate(incomes):
    return sum(calculate_tax(x) for x in incomes) / sum(incomes)

In [16]:
AVERAGE, calculate_tax(AVERAGE)

(44436, 10394.834135626399)

In [17]:
incomes_np = s
incomes = list(s)

In [18]:
avg_in, avg_tax = average_income(incomes), average_tax_rate(incomes)
avg_in, avg_tax

(44418.11980618355, 0.24289954331110122)

In [19]:
%%timeit
# NOTE: don't forget to disable Energy Management on laptops to get better timings!
average_tax_rate(incomes)

2.29 s ± 67.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [22]:
Tpython = 2290

## Making things comparable

In [23]:
import operator

timings = {}

def ratios(**new):
    assert len(new) == 1
    timings.update(**new)
    last = list(new.values())[0]
    print('\n'.join('%10s: %7.2f' % (name, t / last)
                    for name, t in sorted(timings.items(), key=operator.itemgetter(1))))

ratios(python=Tpython)

    python:    1.00


## NumPy slicing

In [24]:
# from Wikipedia:
# =WENN(A1>256303; A1*0,45-16164,53;
#  WENN(A1>54057; A1*0,42-8475,44;
#  WENN(A1>13769; (A1-13769)*((A1-13769)*0,0000022376+0,2397)+939,57;
#  WENN(A1>8820; (A1-8820)*((A1-8820)*0,0000100727+0,14); 0))))

def calculate_tax_numpy_segments(d):
    tax_seg1 = d[(d > 256303)] * 0.45 - 16164.53
    tax_seg2 = d[(d > 54057) & (d <= 256303)] * 0.42 - 8475.44
    seg3 = d[(d > 13769) & (d <= 54057)] - 13769
    seg4 = d[(d > 8820) & (d <= 13769)] - 8820
    prog_seg3 = seg3 * 0.0000022376 + 0.2397
    prog_seg4 = seg4 * 0.0000100727 + 0.14
    return (
        tax_seg1.sum() +
        tax_seg2.sum() +
        (seg3 * prog_seg3 + 939.57).sum() +
        (seg4 * prog_seg4).sum()
    ) / d.sum()


In [25]:
incomes_np.mean(), calculate_tax_numpy_segments(incomes_np)

(44418.11980618259, 0.2428995433111165)

In [26]:
%%timeit
calculate_tax_numpy_segments(incomes_np)

46 ms ± 3 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [27]:
ratios(numpy=61.3)

     numpy:    1.00
    python:   37.36


## NumPy ufunc

In [28]:
calculate_tax_numpy = np.frompyfunc(calculate_tax, 1, 1)

In [29]:
calculate_tax_numpy(incomes_np).sum() / incomes_np.sum()

0.2428995433111065

In [30]:
%%timeit
calculate_tax_numpy(incomes_np).sum() / incomes_np.sum()

569 ms ± 56.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [32]:
ratios(ufunc=569)

     numpy:    0.11
     ufunc:    1.00
    python:    4.02


## Cython

In [33]:
%%cython
# plain copy from Python code above, only renamed functions

def calculate_tax_cy(income):
    if income > 256303:
        return income * 0.45 - 16164.53
    elif income > 54057:
        return income * 0.42 - 8475.44
    elif income > 13769:
        return (income - 13769) * ((income - 13769) * 0.0000022376 + 0.2397) + 939.57
    elif income > 8820:
        return (income - 8820) * ((income - 8820) * 0.0000100727 + 0.14)
    else:
        return 0

def average_income_cy(incomes):
    return sum(incomes) / len(incomes)

def average_tax_rate_cy(incomes):
    return sum(calculate_tax_cy(x) for x in incomes) / sum(incomes)

In [34]:
average_income_cy(incomes), average_tax_rate_cy(incomes)

(44418.11980618355, 0.24289954331110122)

In [35]:
%%timeit
average_tax_rate_cy(incomes)

2.18 s ± 171 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [36]:
ratios(compiled=2180)

     numpy:    0.03
     ufunc:    0.26
  compiled:    1.00
    python:    1.05


## Faster Cython: static types

In [53]:
%%cython -a

cdef double calculate_tax_cy(double income):
    if income > 256303:
        return income * 0.45 - 16164.53
    elif income > 54057:
        return income * 0.42 - 8475.44
    elif income > 13769:
        return (income - 13769) * ((income - 13769) * 0.0000022376 + 0.2397) + 939.57
    elif income > 8820:
        return (income - 8820) * ((income - 8820) * 0.0000100727 + 0.14)
    else:
        return 0

def average_income_cy(incomes):
    return sum(incomes) / len(incomes)

def average_tax_rate_cy(incomes):
    cdef double x
    tax = 0.0
    income_sum = 0.0
    for x in incomes:
        income_sum += x
        tax += calculate_tax_cy(x)
    return tax / income_sum
#     return sum(calculate_tax_cy(x) for x in incomes) / sum(incomes)

In [49]:
average_tax_rate_cy(incomes)

0.24289954331110122

In [50]:
%%timeit
average_tax_rate_cy(incomes)

22.7 ms ± 321 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [51]:
ratios(typed=22.7)

     typed:    1.00
     numpy:    2.70
     ufunc:   25.07
  compiled:   96.04
    python:  100.88


In [57]:
%%cython -a

# SOLUTION

cdef double calculate_tax_cy(double income):
    if income > 256303:
        return income * 0.45 - 16164.53
    elif income > 54057:
        return income * 0.42 - 8475.44
    elif income > 13769:
        return (income - 13769) * ((income - 13769) * 0.0000022376 + 0.2397) + 939.57
    elif income > 8820:
        return (income - 8820) * ((income - 8820) * 0.0000100727 + 0.14)
    else:
        return 0

def average_tax_rate_cy(incomes):
    # return sum(calculate_tax_cy(x) for x in incomes) / sum(incomes)
    cdef double tax = 0, income = 0, x
    for x in incomes:
        income += x
        tax += calculate_tax_cy(x)
    return tax / income


In [58]:
average_tax_rate_cy(incomes)

0.24289954331110122

In [59]:
%%timeit
average_tax_rate_cy(incomes)

24.5 ms ± 2.09 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [60]:
ratios(typed=24.5)

     typed:    1.00
     numpy:    2.50
     ufunc:   23.22
  compiled:   88.98
    python:   93.47


## Exercise: static typing for speed

The function below calculates the circular distance between two points on a globe.

- Speed it up by compiling the computation to plain C in Cython.
- Measure the calculation time before and afterwards.
- Try to avoid type declarations where possible.

In [61]:
import math

def circular_distance(radius, lon1, lat1, lon2, lat2):
    x = math.pi/180.0
    a = (90.0-lat1) * x
    b = (90.0-lat2) * x
    theta = (lon2-lon1) * x
    c = math.acos((math.cos(a)*math.cos(b)) + (math.sin(a)*math.sin(b)*math.cos(theta)))
    return radius*c

print(circular_distance(10, 1.2, 2, 2, 4.3))

0.4249430879322785


In [62]:
%%timeit
circular_distance(10, 1.2, 2, 2, 4.3)

855 ns ± 30 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [110]:
%%cython
# copy and optimise ...
# hint: use "libc.math" from C instead of "math" from Python
# try to use as little type declarations as possible
# import math
cimport libc.math

cdef double circular_distance(double radius, double lon1, double lat1, double lon2, double lat2):
    x = libc.math.pi/180.0
    a = (90.0-lat1) * x
    b = (90.0-lat2) * x
    theta = (lon2-lon1) * x
    c = libc.math.acos((libc.math.cos(a)*libc.math.cos(b)) + (libc.math.sin(a)*libc.math.sin(b)*libc.math.cos(theta)))
    return radius*c

print(circular_distance(10, 1.2, 2, 2, 4.3))

In [111]:
%%timeit
circular_distance(10, 1.2, 2, 2, 4.3)

278 ns ± 8.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


## Faster Cython: processing memory views

`def` is public and can be called from outside the module

`cdef` is a fast C function but cannot be called from outside the module

`cpdef` can be called from outside the module but is till fast

In [100]:
 %%cython -a

cpdef double calculate_tax_cy(double income):
    if income > 256303:
        return income * 0.45 - 16164.53
    elif income > 54057:
        return income * 0.42 - 8475.44
    elif income > 13769:
        return (income - 13769) * ((income - 13769) * 0.0000022376 + 0.2397) + 939.57
    elif income > 8820:
        return (income - 8820) * ((income - 8820) * 0.0000100727 + 0.14)
    else:
        return 0

cimport cython

@cython.boundscheck(False)
@cython.wraparound(False)
def average_tax_rate_memview(double[:] incomes):
    # return sum(calculate_tax_cy(x) for x in incomes) / sum(incomes)
    cdef ssize_t i
    cdef double tax = 0, income = 0, x
    for i in range(incomes.shape[0]):
        x = incomes[i]
        income += x
        tax += calculate_tax_cy(x)
    return tax / income

In [104]:
average_tax_rate_memview(incomes_np)

0.24289954331110122

In [108]:
%%timeit
average_tax_rate_memview(incomes_np)

9.04 ms ± 594 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [109]:
ratios(mviews=9.04)

    mviews:    1.00
     typed:    2.71
     numpy:    6.78
     ufunc:   62.94
  compiled:  241.15
    python:  253.32


In [ ]:
%%cython

# SOLUTION

cpdef double calculate_tax_cy(double income):
    if income > 256303:
        return income * 0.45 - 16164.53
    elif income > 54057:
        return income * 0.42 - 8475.44
    elif income > 13769:
        return (income - 13769) * ((income - 13769) * 0.0000022376 + 0.2397) + 939.57
    elif income > 8820:
        return (income - 8820) * ((income - 8820) * 0.0000100727 + 0.14)
    else:
        return 0

cimport cython

@cython.boundscheck(False)
def average_tax_rate_memview(double[:] incomes):
    cdef unsigned long i
    cdef double tax = 0, income = 0, x
    for i in range(incomes.shape[0]):
        x = incomes[i]
        income += x
        tax += calculate_tax_cy(x)
    return tax / income


In [ ]:
average_tax_rate_memview(incomes_np)

In [ ]:
%%timeit
average_tax_rate_memview(incomes_np)

In [ ]:
ratios(mviews=10.7)

## Exercise: calculate circular distance for NumPy array

In [ ]:
data = 180.0 * np.random.rand(4, 1000) - 90.0
a = data[:2]
b = data[2:]

# Points in a and b: [[longitudes], [latitudes]]
a

In [ ]:
for i in range(3):
    longitude = a[0, i]
    latitude = a[1, i]
    print("Longitude: {:6.3f}, Latitude: {:6.3f}".format(longitude, latitude))
print("...")

In [ ]:
%%cython
# copy your Cython circular_distance() function here to allow for fast C calls
# unpack "points_a" and "points_b" into memory views
# hint: 2D memory views are spelled "dtype[:,:]", e.g. "double[:,:]"
# make sure a and b have the same length in their second dimension
# loop over the range of points in a and apply the function to the points taken from a and b

def calculate_distances(radius, points_a, points_b, output):
    ...
    return output

In [ ]:
output = np.empty(a.shape[1], dtype=np.double)
calculate_distances(100, a, b, output)

In [ ]:
%%timeit
calculate_distances(100, a, b, output)

## Faster Cython: prange

In [139]:
%%cython
# cython: auto_pickle=False
# distutils: extra_compile_args=-fopenmp
# distutils: extra_link_args=-fopenmp

cpdef double calculate_tax_cy(double income) nogil:
    if income > 256303:
        return income * 0.45 - 16164.53
    elif income > 54057:
        return income * 0.42 - 8475.44
    elif income > 13769:
        return (income - 13769) * ((income - 13769) * 0.0000022376 + 0.2397) + 939.57
    elif income > 8820:
        return (income - 8820) * ((income - 8820) * 0.0000100727 + 0.14)
    else:
        return 0

cimport cython

@cython.boundscheck(False)
def average_tax_rate_prange(double[:] incomes):
    cdef unsigned long i
    cdef double tax = 0, income = 0, x
    for i in range(incomes.shape[0]):
        x = incomes[i]
        income += x
        tax += calculate_tax_cy(x)
    return tax / income


In [140]:
average_tax_rate_prange(incomes_np)

0.24289954331110122

In [141]:
%%timeit
average_tax_rate_prange(incomes_np)

9.49 ms ± 578 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [116]:
ratios(prange=6.32)

    prange:    1.00
    mviews:    1.43
     typed:    3.88
     numpy:    9.70
     ufunc:   90.03
  compiled:  344.94
    python:  362.34


In [ ]:
%%cython
# distutils: extra_compile_args=-fopenmp
# distutils: extra_link_args=-fopenmp

# SOLUTION

cpdef double calculate_tax_cy(double income) nogil:
    if income > 256303:
        return income * 0.45 - 16164.53
    elif income > 54057:
        return income * 0.42 - 8475.44
    elif income > 13769:
        return (income - 13769) * ((income - 13769) * 0.0000022376 + 0.2397) + 939.57
    elif income > 8820:
        return (income - 8820) * ((income - 8820) * 0.0000100727 + 0.14)
    else:
        return 0

cimport cython
from cython.parallel cimport prange

@cython.boundscheck(False)
def average_tax_rate_prange(double[:] incomes):
    cdef unsigned long i
    cdef double tax = 0, income = 0, x
    for i in prange(incomes.shape[0], nogil=True, num_threads=4):
        x = incomes[i]
        income += x
        tax += calculate_tax_cy(x)
    return tax / income


In [ ]:
average_tax_rate_prange(incomes_np)

In [ ]:
%%timeit
average_tax_rate_prange(incomes_np)

In [ ]:
ratios(prange=1)

## Exercise: parallelise circular distance calculation with prange()

In [ ]:
%%cython
# copy your Cython circular_distance() function here to allow for fast C calls
# unpack "points_a" and "points_b" into memory views
# hint: 2D memory views are spelled "dtype[:,:]", e.g. "double[:,:]"
# make sure a and b have the same length in their second dimension
# loop over the range of points in a and apply the function to the points taken from a and b
# try different numbers of threads

def calculate_distances_prange(radius, double[:,:] points_a, double[:,:] points_b, output):
    ...
    return output

In [ ]:
%%timeit
calculate_distances_prange(100, a, b, output)

## more ...

In [ ]:
%%cython

cpdef double calculate_tax_cy(double income):
    if income > 256303:
        return income * 0.45 - 16164.53
    elif income > 54057:
        return income * 0.42 - 8475.44
    elif income > 13769:
        return (income - 13769) * ((income - 13769) * 0.0000022376 + 0.2397) + 939.57
    elif income > 8820:
        return (income - 8820) * ((income - 8820) * 0.0000100727 + 0.14)
    else:
        return 0

def average_tax_cy(incomes):
    # return sum(calculate_tax_cy(x) for x in incomes) / sum(incomes)
    cdef double tax = 0, income = 0, x
    for x in incomes:
        income += x
        tax += calculate_tax_cy(x)
    return tax / income


cimport numpy as cnp

def average_tax_numcy(cnp.ndarray[double, ndim=1] incomes):
    cdef long i
    cdef double tax = 0, income = 0, x
    for i in range(incomes.shape[0]):
        x = incomes[i]
        income += x
        tax += calculate_tax_cy(x)
    return income / tax


## Pythran integration

In [145]:
%%cython -a
# cython: np_pythran=True

import numpy as np
cimport numpy as cnp

def calculate_tax_numpy_pythran(cnp.ndarray[double, ndim=1] d):
    tax_seg1 = d[(d > 256303)] * 0.45 - 16164.53
    tax_seg2 = d[(d > 54057) & (d <= 256303)] * 0.42 - 8475.44
    seg3 = d[(d > 13769) & (d <= 54057)] - 13769
    seg4 = d[(d > 8820) & (d <= 13769)] - 8820
    prog_seg3 = seg3 * 0.0000022376 + 0.2397
    prog_seg4 = seg4 * 0.0000100727 + 0.14
    return (
        np.sum(tax_seg1) +
        np.sum(tax_seg2) +
        np.sum(seg3 * prog_seg3 + 939.57) +
        np.sum(seg4 * prog_seg4)
    ) / np.sum(d)


CompileError: command 'x86_64-linux-gnu-gcc' failed with exit status 1

In [ ]:
calculate_tax_numpy_pythran(incomes_np)

In [ ]:
%%timeit
calculate_tax_numpy_pythran(incomes_np)

In [ ]:
ratios(pythran=42)